<a href="https://colab.research.google.com/github/InstitutodaVinci3/Vancouver/blob/master/Producao_Diaria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Resumo dos Trabalhos de Coleta, Limpeza e Entendimento dos dados

##Biblotecas Utilizadas

In [3]:
# importando as bibliotecas básicas e preparando ambiente
import pandas as pd                     # convenção 
import numpy as np                      # convenção 
import random as rd                     # rd não é alias padrão mas sim adotado aqui somente 
# Itens adicionais
import glob
import os
import re

##Coleta e Limpeza dos Dados

###Passo 1 - montagem do drive para colocação dos arquivos recebidos

In [ ]:
# montagem do Google Drive para acessar os arquivos que foram disponibilizados
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Passo 2 - definição da varíavel "Filenames" para ser utilizado no "for" de leitura, de forma a correr todoos os arquivos do tipo TXT existentes no caminho (path)

In [ ]:
# Defifinindo a varíavel FIlenames para ser utilizado no "for" de leitura
path =r'/content/drive/MyDrive/MedMep/Prod_Diaria/'
filenames = glob.glob(path + "/*.txt")
filenames

###Passo 3 - Definição de como capturar a data que está no nome do arquivo
#### É necessário capturar a data inserindo-a em uma coluna adicional para que possamos utilizar os dados de produção de leite em conjunto com os dados de temperatura

In [ ]:
# Ler parte do nome do arquivo
b = re.search(r'(\d+)', '/content/drive/MyDrive/MedMep/Prod_Diaria/Produção diária TXT_20170901T173500.txt')
print(b[1])

#Formatando Data
import datetime
datetime.datetime.strptime("20170901", "%Y%m%d").strftime("%d/%m/%Y")

###Passo 4 - Leitura dos arquivos

In [ ]:
# Lendo os dados já com removendo o cabeçalho e rodapés dos arquivos
# Necessário criar o "Cabeçalho" do DataFrame conforme comando asseguir:
Prod_Dia_df = pd.DataFrame({'Grupo':[], 'Id_Animal':[], 'Num_Lac':[], 'Dias_Lac':[], 'Pico':[], 'Hora_S1':[],
                            'Hora_S2':[], 'Hora_S3':[], 'Durac_S1':[], 'Durac_S2':[], 'Durac_S3':[],
                            'Prod_S1':[], 'Prod_S2':[], 'Prod_S3':[], 'Hora_S1_ont':[], 'Hora_S2_ont':[], 
                            'Hora_S3_ont':[], 'Dura_S1_ont':[], 'Dura_S2_ont':[], "Dura_S3_ont":[],
                            'Prod_S1_ont':[], 'Prod_S2_ont':[], 'Prod_S3_ont':[]})

# Início do looping de leitura dos arquivos
for filename in filenames:

# Leitura dos arquivos já com o novo cabeçalho definido:
  a = pd.read_csv(filename, skiprows=1, skipfooter=2, sep='\t', encoding='latin1', engine='python', decimal=',',   
                  names=['Grupo', 'Id_Animal', 'Num_Lac', 'Dias_Lac', 'Pico', 'Hora_S1', 'Hora_S2', 'Hora_S3',
                         'Durac_S1', 'Durac_S2', 'Durac_S3', 'Prod_S1', 'Prod_S2', 'Prod_S3',
                         'Hora_S1_ont', 'Hora_S2_ont', 'Hora_S3_ont', 'Dura_S1_ont', 'Dura_S2_ont', "Dura_S3_ont",
                         'Prod_S1_ont', 'Prod_S2_ont', 'Prod_S3_ont'])

# Criação da coluna adicional com a data contida no nome do arquivo:
  a['Data']= datetime.datetime.strptime(re.search(r'(\d+)', filename)[1], "%Y%m%d").strftime("%d/%m/%Y") 

# Concatenando os arquivos lidos dentro do looping 
  Prod_Dia_df = pd.concat([Prod_Dia_df, a])
# Fim do looping de leitura

Prod_Dia_df.head()

- Foram disbonibilizados 177 arquivos no formato TXT
- Foi necessário descartar a primeira linha (cabeçalho) de todos os arquivos por falta de padrão nos arquivos enviados
- Foi necessário remover as duas últimas linhas que comtinham comnandos de "end line" e não seriam necessários ao ralizarmos a concatenação dos arquivos
- Foi necessário criar uma nova definição para o cabeçalho do DataFrame
- Foi necessário criar uma nova coluna para a inserção da data da coleta que estava no nome do arquivo
- Os 177 arquivos foram lidos e concatenados gerando um único DataFrame unificado com todos os dados da Produção Diária de leite

In [ ]:
Prod_Dia_df.drop(columns=['Grupo', 'Num_Lac', 'Dias_Lac', 'Pico', 'Durac_S1', 'Durac_S2', 'Durac_S3', 
          'Hora_S1_ont', 'Hora_S2_ont', 'Hora_S3_ont', 'Dura_S1_ont', 'Dura_S2_ont', "Dura_S3_ont",
          'Prod_S1_ont', 'Prod_S2_ont', 'Prod_S3_ont'], inplace=True)
Prod_Dia_df.head()

In [ ]:
# Extraindo as primeiras linhas do arquivo
Prod_Dia_df.head()

In [ ]:
# Extraindo as últimas linhas do arquivo
Prod_Dia_df.tail()

In [ ]:
# Prod_Dia_df.isnull().sum()

In [ ]:
# Exportando dados dos arquivos unificados para Excel
Prod_Dia_df.to_csv("/content/drive/MyDrive/MedMep/Prod_Diaria/Prod_Diaria_Unificado.csv")

##Dashboard

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/InstitutodaVinci3/Vancouver/master/Prod_Diaria_Unificado.csv')
df.head()

,Unnamed: 0,Id_Animal,Hora_S1,Hora_S2,Hora_S3,Prod_S1,Prod_S2,Prod_S3,Data
0,0,653.0,07:47,15:26,,15.84,7.88,,01/09/2017
1,1,2623.0,08:09,15:43,,13.81,7.66,,01/09/2017
2,2,2624.0,07:59,15:35,,12.63,10.54,,01/09/2017
3,3,2662.0,09:31,,,13.64,,,01/09/2017
4,4,3652.0,07:47,15:26,,12.73,6.80,,01/09/2017


In [ ]:
# Producao_S1 = Prod_Dia_df.groupby('Id_Animal').Prod_S1.sum()
# Producao_S1

In [ ]:
# Exportando dados dos arquivos unificados para Excel
# Producao_S1.to_excel('/content/drive/MyDrive/MedMep/Prod_Diaria/Producao-S1.xlsx')